## Get Firmwares

Notebook to download pre-built firmwares published on the micropython download pages


In [1]:
# Below are the requirements for the notebook to run.
# %pip install BeautifulSoup4 requests

In [2]:
import requests
import re
import functools
from urllib.parse import urljoin

from typing import List, Dict
from bs4 import BeautifulSoup

In [3]:
# use functools.lru_cache to avoid needing to download pages multiple times
@functools.lru_cache(maxsize=500)
def get_page(page_url) -> str:
    """Get the HTML of a page and return it as a string."""
    response = requests.get(page_url)
    downloads_html = response.content.decode()
    return downloads_html


@functools.lru_cache(maxsize=500)
def get_board_urls(page_url) -> List[Dict[str, str]]:
    """Get the urls to all the board pages listed on this page."""
    downloads_html = get_page(page_url)
    soup = BeautifulSoup(downloads_html, "html.parser")
    tags = soup.findAll("a", recursive=True, attrs={"class": "board-card"})
    # assumes that all links are relative to the page url
    boards = [tag.get("href") for tag in tags]
    if "?" in page_url:
        page_url = page_url.split("?")[0]
    return [{"board": board, "url": page_url + board} for board in boards]

In [4]:
def firmware_list(board_url: str, base_url: str, ext: str) -> List[str]:
    """Get the urls to all the firmware files for a board."""
    html = get_page(board_url)
    soup = BeautifulSoup(html, "html.parser")
    # get all the a tags
    #   that have a url that starts with `/resources/firmware/`
    #   and ends with a matching extension
    tags = soup.findAll(
        "a",
        recursive=True,
        attrs={"href": re.compile(r"^/resources/firmware/.*\." + ext.lstrip(".") + "$")},
    )
    if "?" in base_url:
        base_url = base_url.split("?")[0]
    links: List = [urljoin(base_url, tag.get("href")) for tag in tags]
    return links

In [5]:
MICROPYTHON_ORG_URL = "https://micropython.org/"
# ports and firmware extensions we are interested in
PORT_FWTYPES = {
    "stm32": ".hex",
    "esp32": ".bin",
    "rp2": ".uf2",
    "samd": ".uf2",
}
# boards we are interested in ( this avoids getting a lot of boards we don't care about)
RELEVANT_BOARDS = [
    "PYBV11",
    "ESP32_GENERIC",
    "ESP32_GENERIC_S3",
    "RPI_PICO",
    "RPI_PICO_W",
    "ARDUINO_NANO_RP2040_CONNECT",
    "PIMORONI_PICOLIPO_16MB",
    "SEEED_WIO_TERMINAL",
]


# The first run takes ~60 seconds to run for 4 ports , all boards
# so it makes sense to cache the results and skip boards as soon as possible
def get_boards(fw_types, RELEVANT_BOARDS):
    board_urls = []
    for port in fw_types.keys():
        download_page_url = f"{MICROPYTHON_ORG_URL}download/?port={port}"
        _urls = get_board_urls(download_page_url)
        # filter out boards we don't care about
        _urls = [board for board in _urls if board["board"] in RELEVANT_BOARDS]
        # add the port to the board urls
        for board in _urls:
            board["port"] = port

        for board in _urls:
            # add a board to the list for each firmware found
            firmwares = firmware_list(board["url"], MICROPYTHON_ORG_URL, fw_types[port])
            for _url in firmwares:
                board["firmware"] = _url
                board["preview"] = "preview" in _url  # type: ignore
                # get version number and optional preview from url
                # 'PYBD_SF2-20231009-v1.22.0-preview.3.ga06f4c8df.hex' -> v1.22.0-preview.3
                # 'PYBD_SF2-20230426-v1.20.0.hex' -> v1.20.0
                ver_match = re.search(r"(\d+\.\d+\.\d+(-\w+.\d+)?)", _url)
                if ver_match:
                    board["version"] = ver_match.group(1)
                else:
                    board["version"] = ""

                board_urls.append(board.copy())
    return board_urls


board_urls = get_boards(PORT_FWTYPES, RELEVANT_BOARDS)

print(f"Found {len(board_urls)} firmwares")

Found 169 firmwares


In [6]:
relevant = [3````````````````
    # and b["port"] in ["esp32", "rp2"]
]
# relevant
print(f"Found {len(relevant)} relevant firmwares")

Found 102 relevant firmwares


In [13]:
# download the relevant files to the firmware folder
import requests
from pathlib import Path

firmware_folder = Path("firmware")
firmware_folder.mkdir(exist_ok=True)

for board in relevant:
    date_re = r"(-\d{8}-)"
    hash_re = r"(.g[0-9a-f]+\.)"
    # remove date from firmware name
    fname = re.sub(date_re, "-", Path(board["firmware"]).name)
    # remove hash from firmware name
    fname = re.sub(hash_re, ".", fname)
    filename = firmware_folder / board["port"] / fname
    filename.parent.mkdir(exist_ok=True)
    if filename.exists():
        # print(f" {filename} already exists, skip download")
        continue
    print(f"Downloading {board['firmware']} to {filename}")
    r = requests.get(board["firmware"], allow_redirects=True)
    with open(filename, "wb") as f:
        f.write(r.content)
    board["filename"] = filename

In [12]:
!dir -l firmware\esp32

 Volume in drive C has no label.
 Volume Serial Number is 30ED-D319

 Directory of c:\develop\MyPython\micropython-stubber\scripts


 Directory of c:\develop\MyPython\micropython-stubber\scripts\firmware\esp32

03-01-2024  11:55    <DIR>          .
19-10-2023  14:24    <DIR>          ..
19-10-2023  14:45         1.494.672 ESP32_GENERIC-D2WD-v1.21.0.bin
19-10-2023  14:45         1.494.832 ESP32_GENERIC-D2WD-v1.22.0-preview.27.bin
30-12-2023  00:28         1.493.840 ESP32_GENERIC-D2WD-v1.22.0-preview.278.bin
30-12-2023  00:28         1.494.112 ESP32_GENERIC-D2WD-v1.22.0-preview.281.bin
30-12-2023  00:28         1.494.176 ESP32_GENERIC-D2WD-v1.22.0-preview.283.bin
30-12-2023  00:28         1.494.144 ESP32_GENERIC-D2WD-v1.22.0-preview.289.bin
19-10-2023  14:45         1.494.832 ESP32_GENERIC-D2WD-v1.22.0-preview.30.bin
19-10-2023  14:45         1.494.832 ESP32_GENERIC-D2WD-v1.22.0-preview.31.bin
19-10-2023  14:45         1.494.832 ESP32_GENERIC-D2WD-v1.22.0-preview.32.bin
30-12-2023  00:28

File Not Found
